In [1]:
import os
import sys

import warnings
warnings.filterwarnings("ignore") 

os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [2]:
# boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\research\akadaner\2021-01-06 (5).xlsx")
boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
boilings = make_boilings_by_groups(boiling_plan_df)
schedule = make_schedule(boilings, start_times={'water': '08:00', 'salt': '07:00'})
frontend = make_frontend(schedule)
draw_excel_frontend(frontend, open_file=True)